# Curvas elípticas

In [4]:
import math
from typing import Tuple

In [5]:
def egcd(x: int, y: int) -> Tuple[int, int, int]:
    """Algoritmo de euclides extendido

    Args:
        x (int)
        y (int)

    Returns:
        (r: int, s:int, t:int) tal que s*x + t*y = r = mdc(x, y)
    """
    r2, r1 = (y, x) if x < y else (x, y)

    s1, s2 = 1, 0
    t1, t2 = 0, 1

    while r2 != 0: 
        q = r1 // r2
        r = r1 % r2 

        r1, s1, t1, r2, s2, t2 = r2, s2, t2, r, s1 - s2*q, t1 - t2*q

    return r1, s1, t1

def modinv(a, b):

    # El pequeño teorema de ferman 
    # Si mcd(a, p) = 1
    # a ^ p - 1 = 1 mod p     
    r, s, _ = egcd(a, b)

    if r != 1:
        raise Exception("Inverse doesn't exist")
    
    return (s % b + b) % b

def inv(y, p):
    return abs(y) * (p - 1) % p

In [6]:
class EllipticCurve:
    
    def __init__(self, a: int, b: int, p: int, G: Tuple[int, int] = None) -> None:
        self.a = a
        self.b = b
        self.p = p 
        self.I = (None, None)
        self.G = G 

        assert(((4 * a ** 3) + (27 * b ** 2)) % p != 0), "No es una curva valida"
    
    
    def __contains__(self, point: Tuple[int, int]) -> bool:
        x, y = point

        if x == y == None: return True
        return pow(y, 2, self.p) == (x**3 + self.a * x + self.b) % self.p


    def __iter__(self):
        yield self.I
        
        for x in range(0, self.p):
            for y in range(0, self.p):
                if (x, y) in self:
                    yield (x, y)
                    y_inv = y * (self.p - 1) % self.p
                    if y_inv != y:
                        yield (x, y_inv)
                    break
    

    def __len__(self):
        return len([p for p in self])
    

    def hasse(self):
        """Teorema de Hasse. Define el intervalo 

        p + 1 - 2 raiz(p), p + 1+ 2*raiz(p)
        """

        lower = self.p + 1 - 2 * math.sqrt(self.p)
        upper = self.p + 1 + 2 * math.sqrt(self.p)
        
        return lower <= len(self) <= upper, (lower, upper)

    def add(self, P: Tuple[int, int], Q: Tuple[int, int] = None) -> Tuple[int, int]:
        """Adición de puntos en la curva P-256."""
        Q = P if Q is None else Q
        
        if not (P in self and Q in self):
            raise ValueError("One or more points are not on the curve")

        # Identidad 
        if P == (None, None):  # P es el punto en el infinito
            return Q
        
        if Q == (None, None):  # Q es el punto en el infinito
            return P 

        x1, y1 = P
        x2, y2 = Q

        # Inverso aditivo: P + (-P) = I
        if x1 == x2 and (y1 == -y2 % self.p):
            return (None, None)  # Retorna el punto en el infinito
        
        # Pendiente de la recta que pasa por P y Q
        if x1 != x2:
            # Caso general: P + Q
            s = ((y2 - y1) * pow(x2 - x1, -1, self.p)) % self.p
            x3 = (s ** 2 - x1 - x2) % self.p
        else: 
            # Caso especial: P = Q
            s = ((3 * x1 ** 2 + self.a) * pow(2 * y1, -1, self.p)) % self.p
            x3 = (s ** 2 - 2 * x1) % self.p 
            
        y3 = (s * (x1 - x3) - y1) % self.p 

        return (x3, y3)

    def mult(self, k: int, P: Tuple[int, int]) -> Tuple[int, int]:
        Q = P if k >= 0 else (P[0], inv(P[1]))
        # Inicialización de T
        T = Q if k & 1 else self.I
        k >>= 1

        # Multiplicación a escalar
        while k:
            Q = self.add(Q, Q)
            if k & 1:
                T = self.add(T, Q)
            k >>= 1

        return T

In [7]:
p = 29
a = 4
b = 20 

ec = EllipticCurve(a, b, p)

points = [
    (None, None), (0, 7), (0, 22), (1, 5), (1, 24), (2, 6), (2, 23), (3, 1), (3, 28), (4, 10), (4, 19), (5, 7), (5, 22), (6, 12), (6, 17), (8, 10), (8, 19), (10, 4), (10, 25), (13, 6), (13, 23), (14, 6), (14, 23), (15, 2), (15, 27), (16, 2), (16, 27), (17, 10), (17, 19), (19, 13), (19, 16), (16, 27), (17, 10), (19, 13), (19, 16), (20, 3), (20, 26), (24, 7), (24, 22), (27, 2), (27, 27)
]

for p in points: 
    assert(p in ec), f"Failed for point {p}" 

    
for p in ec:
    assert(p in points), "Wtf point"

assert(len(ec) == 37)
assert(ec.add((1, 5)) == (4, 19))
assert(ec.add((1, 5), (20, 3)) == (15, 27))
assert(ec.add((1, 5), (1, 24)) == (None, None))

ec.hasse() # (30 - 10.7703296143, 30 + 10.7703296143)


(True, (19.22967038573099, 40.77032961426901))

In [8]:
P = (4, 19)
T = ec.I 
T = ec.add(T, P)
T = ec.add(T, P)
T = ec.add(T, P)

In [9]:
p = 131
a = 50
b = 107

ec = EllipticCurve(a, b, p)

points = [(None, None), (0, 32),(0, 99),(1, 17),(1, 114),(2, 52),(2, 79),(4, 41),(4, 90),(5, 58),(5, 73),(6, 19),(6, 112),(8, 44),(8, 87),(9, 32),(9, 99),(10, 64),(10, 67),(12, 48),(12, 83),(16, 5),(16, 126),(18, 17),(18, 114),(22, 14),(22, 117),(23, 18),(23, 113),(27, 7),(27, 124),(30, 7),(30, 124),(34, 34),(34, 97),(35, 42),(35, 89),(38, 5),(38, 126),(42, 61),(42, 70),(43, 46),(43, 85),(44, 30),(44, 101),(46, 24),(46, 107),(47, 63),(47, 68),(48, 35),(48, 96),(50, 12),(50, 119),(52, 1),(52, 130),(54, 53),(54, 78),(56, 25),(56, 106),(57, 54),(57, 77),(59, 43),(59, 88),(60, 59),(60, 72),(61, 44),(61, 87),(62, 44),(62, 87),(63, 9),(63, 122),(64, 62),(64, 69),(65, 0),(67, 13),(67, 118),(69, 51),(69, 80),(70, 51),(70, 80),(73, 6),(73, 125),(74, 7),(74, 124),(75, 47),(75, 84),(77, 5),(77, 126),(78, 4),(78, 127),(82, 2),(82, 129),(84, 31),(84, 100),(86, 2),(86, 129),(87, 10),(87, 121),(88, 60),(88, 71),(90, 20),(90, 111),(91, 0),(92, 43),(92, 88),(93, 53),(93, 78),(94, 2),(94, 129),(95, 11),(95, 120),(98, 36),(98, 95),(99, 42),(99, 89),(100, 59),(100, 72),(101, 54),(101, 77),(102, 59),(102, 72),(104, 54),(104, 77),(106, 0),(108, 26),(108, 105),(111, 43),(111, 88),(112, 17),(112, 114),(114, 34),(114, 97),(115, 53),(115, 78),(116, 49),(116, 82),(118, 50),(118, 81),(120, 45),(120, 86),(121, 21),(121, 110),(122, 32),(122, 99),(123, 51),(123, 80),(126, 16),(126, 115),(127, 57),(127, 74),(128, 42),(128, 89)]


for p in ec:
    assert(p in points), f"wtf point {p}"

for p in points:
    assert(p in ec), f"wtf point {p}"

assert(len(ec) == len(points)), "Wrong len"

ec.hasse()

(True, (109.10895371548081, 154.8910462845192))

In [10]:
p = 167
a = -3
b = 64

ec = EllipticCurve(a, b, p)

points = [(None, None),(0, 8),(0, 159),(1, 79),(1, 88),(2, 20),(2, 147),(4, 28),(4, 139),(5, 72),(5, 95),(9, 76),(9, 91),(10, 52),(10, 115),(14, 42),(14, 125),(15, 80),(15, 87),(19, 55),(19, 112),(21, 54),(21, 113),(24, 17),(24, 150),(26, 19),(26, 148),(27, 36),(27, 131),(30, 33),(30, 134),(31, 6),(31, 161),(32, 2),(32, 165),(34, 45),(34, 122),(36, 55),(36, 112),(40, 22),(40, 145),(41, 15),(41, 152),(53, 73),(53, 94),(56, 50),(56, 117),(57, 65),(57, 102),(61, 57),(61, 110),(62, 8),(62, 159),(66, 17),(66, 150),(69, 82),(69, 85),(70, 62),(70, 105),(71, 81),(71, 86),(72, 39),(72, 128),(77, 17),(77, 150),(79, 7),(79, 160),(83, 63),(83, 104),(84, 0),(85, 83),(85, 84),(86, 60),(86, 107),(90, 29),(90, 138),(91, 5),(91, 162),(92, 16),(92, 151),(94, 83),(94, 84),(98, 77),(98, 90),(99, 56),(99, 111),(101, 29),(101, 138),(104, 47),(104, 120),(105, 8),(105, 159),(107, 26),(107, 141),(111, 48),(111, 119),(112, 55),(112, 112),(113, 57),(113, 110),(117, 56),(117, 111),(118, 56),(118, 111),(119, 62),(119, 105),(120, 16),(120, 151),(122, 16),(122, 151),(124, 41),(124, 126),(128, 67),(128, 100),(133, 21),(133, 146),(135, 25),(135, 142),(138, 12),(138, 155),(140, 1),(140, 166),(143, 29),(143, 138),(144, 18),(144, 149),(145, 62),(145, 105),(151, 58),(151, 109),(154, 54),(154, 113),(155, 83),(155, 84),(157, 51),(157, 116),(159, 54),(159, 113),(160, 57),(160, 110),(161, 37),(161, 130),(162, 11),(162, 156),(163, 43),(163, 124),(165, 79),(165, 88),(166, 20),(166, 147),
]

for p in ec:
    assert(p in points), f"wtf point {p}"

for p in points:
    assert(p in ec), f"wtf point {p}"
    
assert(len(ec) == 150), "Wrong len"
ec.hasse() # 142.154, 193.846

(True, (142.15430403335984, 193.84569596664016))

## Curva P256

In [11]:
class CurveP256(EllipticCurve):

    def __init__(self) -> None:
        a = -3 
        b = 41058363725152142129326129780047268409114441015993725554835256314039467401291
        p = 2**256 - 2**224 + 2**192 + 2**96 - 1
        
        x = 48439561293906451759052585252797914202762949526041747995844080717082404635286
        y = 36134250956749795798585127919587881956611106672985015071877198253568414405109
        
        G = (x, y)

        super().__init__(a, b, p, G)
        
        self.q = 2**256 - 2**224 + 2**192 - 89188191075325690597107910205041859247 

In [12]:
P = (5139147556, 84992513513819837562679639329935059290744880011754896240095105442438501460580)
Q = (957, 106068017933963908069108378434410615354534760345589368572553274269533297279953)
R = (39147563, 86800512523116406738886579531519493100624547860931014107623369720327567099304)

P256 = CurveP256()

assert(P256.add(P, Q) == (70080563902474660685014643574954286003305136526954517743428749117055603385364, 102817281760810435925299871406466683251351115534973220072977695964583963694190))


assert(P256.mult(17, P) ==   (6767482609142533773727106561141486021988565046507560472655419434998143805142, 111849450923862212349083160889673641820507651447758924512379621060157659132486))

assert(P256.add(P, P256.mult(29, Q)) == (85770230561767378375651721422453530671836821484546392654290517587029149330503, 109289525563572793074808101103724966387806510899333588476506211211325308580228))
      
assert(P256.add(P, P256.add(Q, R)) == (10078325695723211088410985734864258926108196424648752194418952354939491493398, 83162779257944430915141041076225903572090104654488309266745131800763293112040))

assert(P256.add(P256.mult(5, P), P256.add(P256.mult(47, Q), P256.mult(131, R))) == (60003140733697602815862847258574814281333745187497680713195857338569721153623, 113621516247140099510908544142253158409830215142911456940902986653313431422677))

## Diffie-Hellman 

In [13]:
import random

In [17]:
class DiffieHellman:

    def get(self, curve, q):
        if (curve.G == None):
            raise ValueError("La curva no tiene generador.")
        
        return self.Alice(curve, q), self.Bob(curve, q)

    class Alice: 

        def __init__(self, curve, q):
            self.q = q
            self.curve = curve 

        def send(self, a: int = None):
            a = random.randrange(2, self.q - 1) if a == None else a 
            U = self.curve.mult(a, self.curve.G)
            self.a = a 
            return U

        def recive(self, V: Tuple[int, int]):
            W = self.curve.mult(self.a, V)
            self.W = W 
    
    class Bob:

        def __init__(self, curve, q):
            self.q = q 
            self.curve = curve 
        
        def recive(self, U: Tuple[int, int], b: int = None):
            b = random.randrange(2, self.q - 1) if b == None else b

            V = self.curve.mult(b, self.curve.G)
            W = self.curve.mult(b, U)
            self.W = W
            return V
            

In [20]:
q = 2**256 - 2**224 + 2**192 - 89188191075325690597107910205041859247 

alpha = 8043
beta = 66569451221028833605888206012592990107761917366990348285636907733675199640869

M = (681515878241, 42867525214630099613645669273136473583026733865738235463867590073359574198973)

curve = CurveP256()

alice = DiffieHellman.Alice(curve, q)
bob = DiffieHellman.Bob(curve, q)

U = alice.send()
V = bob.recive(U)
alice.recive(V)
assert(alice.W == bob.W)

## ElGamal sobre curvas eliptivas

In [ ]:
import random

In [ ]:
class ElGamal_EllipticCurve:
    
    def __init__(self, curve: EllipticCurve, q: int = None) -> None:
        if (curve.G == None):
            raise ValueError("La curva no tiene generador.")

        self.curve = curve
        self.q = q


    def G(self, a = None) -> Tuple[Tuple[int, int], int]:
        a = random.randrange(2, self.q) if a == None else a
        U = self.curve.mult(a, self.curve.G)
        return U, a 


    def E(self, Pk: Tuple[int, int], M: Tuple[int, int], b: int = None, debug_log: bool = False):
        b = random.randrange(2, self.q) if b == None else b
        if debug_log: print(f"beta = {b}")

        V = self.curve.mult(b, self.curve.G)
        if debug_log: print(f"v = {V}")

        W = self.curve.mult(b, Pk)
        
        C = self.curve.add(M, W)
        if debug_log: print(f"c = {C}")

        return V, C
    

    def D(self, sk: int, V: int, C: int) -> int:
        W = self.curve.mult(sk, V)
        M = self.curve.add(C, (W[0], inv(W[1], self.curve.p)))
        return M

In [ ]:
q = 2**256 - 2**224 + 2**192 - 89188191075325690597107910205041859247 

alpha = 8043
beta = 66569451221028833605888206012592990107761917366990348285636907733675199640869

M = (681515878241, 42867525214630099613645669273136473583026733865738235463867590073359574198973)

curve = CurveP256()
egP256 = ElGamal_EllipticCurve(curve, q)
Pk, sk = egP256.G(alpha)

assert((Pk, sk) == ((111799829596708300766184347484227372609212980373099566904718736416741928372155, 16113508833059441685350581980088164608160061680445658675441448021238419463670),8043))

(V, C) = egP256.E(Pk, M, b = beta)
assert((V, C) == ((67600389177367027958880505329001218911198820158050052020917398066184915452697, 15585948642504606242912978362059634695851845577015490429642291473142458963401), (85283259038181720772938993814478178376409244351401717659250632910327642310995, 42144723331063680933579200498711580440199831076940199774822685779637332671033)))
assert(egP256.D(sk, V, C) == M)

(V, C) = egP256.E(Pk, M, debug_log = True)
assert(egP256.D(sk, V, C) == M)

beta = 6292296390847819873472920194228056694305816654900896623984775991780269331280
v = (61932790441964176691414102555355973299451383869990383141579608837082699355761, 19252305924548217982430211133306194015857335949220738440168944774906223761674)
c = (110148751766248041623115358695953623861972820280156200801019909786253170355497, 13597353763826944907187929414839853239541939677448720362598275979512226090138)
